In [1]:
import sys
sys.path.append("../")

import QuantLib as ql
import pandas as pd
from datetime import datetime, timedelta 
from pandas.tseries.offsets import CustomBusinessDay 
from pandas.tseries.holiday import USFederalHolidayCalendar

from CurvyCUSIPs.CurveInterpolator import GeneralCurveInterpolator
from CurvyCUSIPs.CurveDataFetcher import CurveDataFetcher
from CurvyCUSIPs.USTs import USTs
from CurvyCUSIPs.S490Swaps import S490Swaps
from CurvyCUSIPs.S490Swaptions import S490Swaptions 
from CurvyCUSIPs.utils.regression_utils import run_multiple_linear_regression_df, run_basic_linear_regression_df, plot_residuals_timeseries

from CurvyCUSIPs.HedgeHog.beta import beta_estimates
from CurvyCUSIPs.HedgeHog.usts import dv01_neutral_curve_hedge_ratio, dv01_neutral_butterfly_hedge_ratio
import CurvyCUSIPs.HedgeHog.swaps as hh_swaps

from CurvyCUSIPs.Spectral.pca_residuals import PCAGridResiduals

import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
config = dotenv_values(env_path)

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
plt.style.use('ggplot')
params = {'legend.fontsize': 'large',
        'figure.figsize': (15, 9),
        'axes.labelsize': 'large',
        'axes.titlesize':'large',
        'xtick.labelsize':'large',
        'ytick.labelsize':'large'}
pylab.rcParams.update(params)

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

In [2]:
curve_data_fetcher = CurveDataFetcher(use_ust_issue_date=True, fred_api_key=config["FRED_API_KEY"])

In [3]:
s490_swaps = S490Swaps(s490_curve_db_path=r"..\db\nyclose_sofr_ois", curve_data_fetcher=curve_data_fetcher)
s490_swaptions = S490Swaptions(s490_swaps)

..\db\nyclose_sofr_ois is behind --- cd into 'scripts' and run 'update_sofr_ois_db.py' to update --- most recent date in db: 2025-01-08 00:00:00
..\db\s490_swaption_atm_vol is behind --- cd into 'scripts' and run 'update_atm_swaption_vol.py' to update --- most recent date in db: 2025-01-07 00:00:00
..\db\s490_swaption_vol_cube is behind --- cd into 'scripts' and run 'update_atm_swaption_vol.py' to update --- most recent date in db: 2025-01-08 00:00:00


In [4]:
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 5, 31)
bdates = pd.date_range(start=start_date, end=end_date, freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))

In [9]:
fwd_tenors = ["1M", "3M", "6M", "9M", "12M", "18M", "2Y", "3Y", "4Y", "5Y", "6Y", "7Y", "8Y", "9Y", "10Y"]
fwd_grid_dict, ql_curves = s490_swaps.s490_nyclose_fwd_curve_matrices(
    start_date=start_date,
    end_date=end_date,
    ql_piecewise_method="logLinearDiscount",
    fwd_tenors=fwd_tenors,
)

Building Implied Fwd Curves...: 100%|██████████| 106/106 [00:01<00:00, 104.64it/s]


In [14]:
swap_pca_residuals = PCAGridResiduals(timeseries_grids=fwd_grid_dict)
swap_pca_results = swap_pca_residuals.runner(run_on_indy_cols=True, n_jobs=-1, remove_underlying_tenors=["1D", "1W", "2W", "3W", "1M", "2M", "3M", "4M", "5M", "6M", "9M"])

CLEANING UP PCA ON INDY COLS RESULTS...: 100%|██████████| 16/16 [00:00<00:00, 105.18it/s]


In [ ]:
swap_pca_results["rich_cheap_residual_zscore_timeseries_dict"][end_date].style.background_gradient(cmap="RdYlGn", axis=0)

,Spot,1M Fwd,3M Fwd,6M Fwd,9M Fwd,12M Fwd,18M Fwd,2Y Fwd,3Y Fwd,4Y Fwd,5Y Fwd,6Y Fwd,7Y Fwd,8Y Fwd,9Y Fwd,10Y Fwd
Tenor,,,,,,,,,,,,,,,,
12M,-1.040209,-1.835527,-1.205542,0.231379,0.546212,1.367060,0.731288,-0.218310,-0.472606,0.640738,-0.049422,1.146137,0.253570,-0.473324,0.202114,0.305963
18M,1.608636,1.989654,2.218557,-0.174856,0.027016,-1.197149,-1.092425,-0.485032,0.317939,0.236636,0.763381,1.282878,-0.350780,0.403933,0.037324,0.398478
2Y,0.861592,1.699912,0.768468,-0.266037,-0.865043,-1.592863,-0.770468,-0.326543,0.769620,-0.183186,0.654476,1.121117,-0.329350,0.492283,-0.091508,0.443357
3Y,0.311493,0.660255,0.095656,-0.412213,-0.710752,-0.913546,0.121130,0.762341,0.282169,0.393318,0.265936,-0.807980,0.318746,0.338421,-0.101982,-0.766822
4Y,-0.560340,-0.824375,-0.403757,-0.000133,0.313006,0.222254,0.570370,0.595197,0.481061,0.081896,-0.843350,-0.711788,0.396650,0.082387,-0.447622,-0.722165
5Y,0.262419,-0.537883,0.350658,0.843045,0.751472,0.467372,0.725551,0.606271,0.357412,-0.787421,-0.467174,-0.872965,0.226370,-0.397637,-0.755515,-0.709400
6Y,0.126402,-0.728492,0.177158,0.744629,0.616360,0.651543,0.740560,0.442917,-0.602695,-0.384534,-0.620050,-0.653139,0.017569,-0.753858,-0.655697,-0.543139
7Y,-0.423814,-0.922020,-0.030298,0.950286,0.835730,0.584904,0.149151,-0.429455,-0.312084,-0.622098,-0.501301,-1.242002,-0.255837,-0.686012,-0.381032,-0.120150
8Y,-0.636010,-0.520595,0.106335,0.412207,0.141646,0.052001,-0.325189,-0.263881,-0.566493,-0.591501,-0.941434,-1.186865,-0.324761,-0.493102,0.057663,0.522614


In [22]:
atm_vol_surfaces_dict = s490_swaptions.get_vol_surfaces(start_date=start_date, end_date=end_date)

In [24]:
swaption_pca_residuals = PCAGridResiduals(timeseries_grids=atm_vol_surfaces_dict)
swaption_pca_results = swaption_pca_residuals.runner(n_jobs=-1, run_on_level_changes=True)

In [25]:
swaption_pca_results["rich_cheap_residual_zscore_timeseries_dict"][end_date].style.background_gradient(cmap="RdYlGn_r", axis=0)

,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,15Y,20Y,25Y,30Y
Expiry,,,,,,,,,,,,,,
1M,-0.976289,0.330465,0.449639,0.374406,0.205061,0.454053,0.675044,0.846236,0.972872,1.062446,0.936090,0.737010,0.528869,0.352384
3M,-1.042606,-0.585515,-0.670824,-0.555992,-0.390756,-0.262348,-0.117197,0.048904,0.225915,0.401965,0.357958,0.248587,0.121790,0.020168
6M,-0.382798,0.034643,0.294070,0.285228,0.211166,0.409131,0.566037,0.682541,0.800035,0.876746,0.931277,0.910096,0.786564,0.613567
9M,-0.074070,0.159745,0.328864,0.153864,-0.061774,0.150660,0.435295,0.599896,0.733049,0.766137,0.656657,0.445694,0.182572,-0.020046
1Y,0.286482,0.202724,0.253835,0.005804,-0.325004,-0.176004,0.017595,0.044099,0.068565,0.086205,-0.095074,-0.248503,-0.374449,-0.476457
2Y,0.612573,0.586709,0.586251,0.483762,0.273041,0.045179,-0.197196,-0.423330,-0.641278,-0.833760,-1.005822,-1.126353,-1.210773,-1.254949
3Y,0.691477,0.349271,0.325854,0.231479,0.085174,-0.128284,-0.388251,-0.685697,-0.992797,-1.273257,-1.516697,-1.665314,-1.729941,-1.740106
4Y,0.679424,0.155023,0.110260,0.155318,-0.060819,-0.400151,-0.581143,-0.804886,-1.124319,-1.496533,-1.825926,-1.963179,-1.964688,-1.935674
5Y,0.601984,-0.171277,-0.246945,-0.271065,-0.308414,-0.521960,-0.762248,-1.021882,-1.260139,-1.432901,-1.743126,-1.908252,-1.930875,-1.894877
